# Setting Up Environment

In [3]:
import time
import pandas as pd
import re
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
import datetime as dt
import numpy as np

import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [15]:
def calculate_best_bet_construct():
    
    # Internal functions

    def calculate_odds_internal(odds):
        if odds<0:
            return (abs(odds)/(abs(odds)+100))
        if odds>0:
            return (100/(odds+100))
    def calculate_bets_internal(row, diff):
        bet = 0
        
        if row.Prediction_GB_Winner - calculate_odds_internal(row.Fighter_1_Odds) >= diff:
            bet = 100
        if (1.0 - row.Prediction_GB_Winner) - calculate_odds_internal(row.Fighter_2_Odds) >= diff:
            bet = 100
        
        return bet
    def calculate_payoff_and_result_internal(row):
        if row.Bet > 0:
            # Calculating Payoff
            if row.Predicted_Result_GB == 1:
                if row.Fighter_1_Odds>0:
                    payoff = (row.Fighter_1_Odds/100)*row.Bet
                else:
                    payoff = row.Bet/((abs(row.Fighter_1_Odds)/100))
            else:
                if row.Fighter_2_Odds>0:
                    payoff = (row.Fighter_2_Odds/100)*row.Bet
                else:
                    payoff = row.Bet/((abs(row.Fighter_2_Odds)/100))
            # Calculating Bet Result
            if row.Predicted_Result_GB == row.result_y:
                bet_result = payoff
            else:
                bet_result = -(row.Bet)
        else:
            bet_result = 0
        return bet_result

    # Setting up data 

    # Joining predictions to table w/ results and getting result
    predictions = pd.read_csv('mma_data_predictions.csv', index_col = 0)
    data = pd.read_csv('mma_data.csv', index_col = 0)
    data = data[data.result >= 0]
    results_data = data[['fighter_1', 'fighter_2', 'result', 'KO_OVR', 'SUB_OVR']]
    odds_data = pd.read_csv('mma_data_odds.csv', index_col = 0)
    merged = predictions.merge(results_data, on = ['fighter_1', 'fighter_2'])
    # Winner results
    merged['Predicted_Result_RF'] = merged.Prediction_RF_Winner.apply(lambda x: 1 if x > 0.5 else 0)
    merged['Predicted_Result_GB'] = merged.Prediction_GB_Winner.apply(lambda x: 1 if x > 0.5 else 0)
    merged['Accurate_RF'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Result_RF else 0, axis = 1)
    merged['Accurate_GB'] = merged.apply(lambda x: 1 if x.result_y == x.Predicted_Result_GB else 0, axis = 1)
    # Sub results
    merged['Predicted_Sub_RF'] = merged.Prediction_RF_SUB.apply(lambda x: 1 if x > 0.5 else 0)
    merged['Predicted_Sub_GB'] = merged.Prediction_GB_SUB.apply(lambda x: 1 if x > 0.5 else 0)
    merged['Accurate_RF_SUB'] = merged.apply(lambda x: 1 if x.SUB_OVR_y == x.Predicted_Sub_RF else 0, axis = 1)
    merged['Accurate_GB_SUB'] = merged.apply(lambda x: 1 if x.SUB_OVR_y == x.Predicted_Sub_GB else 0, axis = 1)
    # KO Results
    merged['Predicted_KO_RF'] = merged.Prediction_RF_KO.apply(lambda x: 1 if x > 0.5 else 0)
    merged['Predicted_KO_GB'] = merged.Prediction_GB_KO.apply(lambda x: 1 if x > 0.5 else 0)
    merged['Accurate_RF_KO'] = merged.apply(lambda x: 1 if x.KO_OVR_y == x.Predicted_KO_RF else 0, axis = 1)
    merged['Accurate_GB_KO'] = merged.apply(lambda x: 1 if x.KO_OVR_y == x.Predicted_KO_GB else 0, axis = 1)
    # Getting all the relevant data in one place for bet constructs
    odds_data = odds_data[['fighter_1', 'fighter_2', 'Fighter_1_Odds', 'Fighter_2_Odds']]
    profit_df = merged.merge(odds_data, on = ['fighter_1', 'fighter_2'])
    profit_df = profit_df[(profit_df.Fighter_1_Odds!=0) & (profit_df.Fighter_2_Odds!=0)]

    # Determining best bet construct

    best_diff = 0
    best_profit = 0
    best_fight_number = 0
    for i in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
        profit_df['Bet'] = profit_df.apply(calculate_bets_internal, diff = i, axis = 1)
        profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result_internal, axis = 1)
        print(f'With a cutoff of {i}, betting results are {profit_df.Bet_Result.sum()}')
        if float(profit_df.Bet_Result.sum()) > best_profit:
            best_diff = i
        if profit_df.Bet_Result.sum() > best_profit:
            best_profit = profit_df.Bet_Result.sum()
    # Veteran fights only
    best_profit = 0
    profit_df['Bet'] = profit_df.apply(calculate_bets_internal, diff = best_diff, axis = 1)
    profit_df['Bet_Result'] = profit_df.apply(calculate_payoff_and_result_internal, axis = 1)
    for num_fights in [0, 5, 10, 15, 20, 25]:
        profit_df['Fights_1'] = profit_df.wins_1 + profit_df.losses_1
        profit_df['Fights_2'] = profit_df.wins_2 + profit_df.losses_2
        test = profit_df[(profit_df.Fights_1 > num_fights) | (profit_df.Fights_2 > num_fights)]
        results = test.Bet_Result.sum()
        print(f'For a {num_fights} fight minimum, the model returns {results}')
        if results > best_profit:
            best_fight_number = num_fights
        if results > best_profit:
            best_profit = results
        
    return best_diff, best_fight_number

In [16]:
print(calculate_best_bet_construct())

With a cutoff of 0.05, betting results are 540.2536891519786
With a cutoff of 0.1, betting results are 504.1103432505683
With a cutoff of 0.15, betting results are 60.941700220577815
With a cutoff of 0.2, betting results are -18.22751322751327
With a cutoff of 0.25, betting results are -125.0
With a cutoff of 0.3, betting results are 75.0
With a cutoff of 0.35, betting results are 0
With a cutoff of 0.4, betting results are 0
With a cutoff of 0.45, betting results are 0
With a cutoff of 0.5, betting results are 0
For a 0 fight minimum, the model returns 540.2536891519786
For a 5 fight minimum, the model returns 540.2536891519786
For a 10 fight minimum, the model returns 393.3909440539395
For a 15 fight minimum, the model returns 372.5360536868542
For a 20 fight minimum, the model returns 129.6515546159143
For a 25 fight minimum, the model returns -527.6570708866911
(0.05, 0)
